In [7]:
import imaplib
import email
# Dependencies
import urllib.request
from lxml import html
from os import getcwd
from os.path import join
from bs4 import BeautifulSoup as bs
import io
from credentials import details

mail = imaplib.IMAP4_SSL('imap.gmail.com')
# imaplib module implements connection based on IMAPv4 protocol
# mail.login('nagaP.stocks@gmail.com', 'stocks_2020')
mail.login(details[0],details[1])

('OK', [b'nagap.stocks@gmail.com authenticated (Success)'])

In [8]:
# print(details[0])

In [9]:
mail.list() # Lists all labels in GMail
mail.select('inbox') # Connected to inbox.

('OK', [b'7'])

In [10]:
result, data = mail.uid('search', 'From "easyguru@gmail.com" Subject "Fwd: Fly Morning News"', "ALL")
i = len(data[0].split())
latest_email_uid = data[0].split()[i-1] 
result, email_data = mail.uid('fetch', latest_email_uid, '(RFC822)')
raw_email = email_data[0][1]
raw_email_string = raw_email.decode('utf-8')
email_message = email.message_from_string(raw_email_string)
for part in email_message.walk():
    if part.get_content_type() == "text/html": # ignore attachments/html
        body = part.get_payload(decode=True)
        text=body.decode("utf-8")
soup = bs(text, 'html.parser')

In [11]:
import pandas as pd
l=[]
for tick in soup.select('a[href*="/news.php?symbol"]') :
        titles = tick.parent.parent.parent.select('a[href^="https://thefly.com/permalinks/entry.php"] span')
        if(tick.text.find(",")==-1 and len(titles)>0):
            dict1 ={}
            dict1["Tick"]=tick.text 
            dict1["News"]=titles[0].text
            l.append(dict1)
flydf =pd.DataFrame(l)
flydf.head(10)  

,News,Tick
0,Neon Therapeutics upgraded to Equal Weight fro...,NTGN
1,Kansas City Southern price target raised to $1...,KSU
2,Envision Solar announces further orders throug...,EVSI
3,"Hemp, Inc. supports House legislation that pro...",HEMP
4,Shake Shack price target lowered to $82 from $...,SHAK
5,Akerna completes acquisition of majority inter...,KERN
6,Regions Financial upgraded to Buy from Neutral...,RF
7,"PetMed Express reports Q3 EPS 34c, two est. 30c",PETS
8,Uber's sale of Indian food delivery business '...,UBER
9,CD Projekt downgraded to Hold from Buy at Jeff...,OTGLY


In [7]:
group_ticker=flydf.groupby("Tick").count()
group_ticker=group_ticker.reset_index()
group_ticker.head()

,Tick,News
0,$COMP,1
1,$DJT,1
2,$DJU,1
3,AA,2
4,AAPL,4


In [8]:
sort_ticker=group_ticker.sort_values("News",ascending=False)
sort_ticker.head()

,Tick,News
44,CFG,6
188,STT,5
4,AAPL,4
176,SLB,4
180,SNAP,3


In [9]:
merged_df=pd.merge(flydf,sort_ticker,on=["Tick"])
merged_df.head()

,News_x,Tick,News_y
0,Domino's Pizza price target raised to $315 fro...,DPZ,1
1,Oportun Financial downgraded to Neutral on val...,OPRT,2
2,Oportun Financial downgraded to Neutral from O...,OPRT,2
3,Essent upgraded to Overweight on 'compelling' ...,ESNT,2
4,Essent Group upgraded to Overweight from Neutr...,ESNT,2


In [10]:
final_tickerdf=merged_df.rename(columns={"News_x": "TickerNews", "News_y": "Count"})
final_tickerdf
final_tickerdf=final_tickerdf.sort_values(["Count","Tick"],ascending=False)
final_tickerdf.head()
# final_tickerdf.to_csv("todayflymorningnews.csv",index=False)


,TickerNews,Tick,Count
226,Citizens Financial CEO reports 'strength in ou...,CFG,6
227,Citizens Financial reports Q4 net interest mar...,CFG,6
228,Citizens Financial reports Q4 underlying EPS 9...,CFG,6
229,Citizens Financial raises quarterly dividend 8...,CFG,6
230,Citizens Financial declares an 8% increase in ...,CFG,6


In [11]:
for index,news in merged_df.iterrows():
       print(news[0])
#         if news[0] == "raises":
#             print(news[0])
    

Domino's Pizza price target raised to $315 from $305 at Maxim
Oportun Financial downgraded to Neutral on valuation at JPMorgan
Oportun Financial downgraded to Neutral from Overweight at JPMorgan
Essent upgraded to Overweight on 'compelling' outlook at JPMorgan
Essent Group upgraded to Overweight from Neutral at JPMorgan
Act II Global Acquisition management to meet with Odeon Capital
Act II Global Acquisition management to meet with Odeon Capital
DJ Transportation Average: Pivot points
DJ Utility Average: Pivot points
S&P Global downgraded at Raymond James on balanced risk/reward
S&P Global downgraded to Market Perform from Outperform at Raymond James
Dow Jones Industrial Average: Pivot points
iShares Transportation Average: Pivot points
Technology Select Sector SPDR: Pivot points
Energy Select Sector SPDR: Pivot points
Consumer Discretionary Sector SPDR: Pivot points
S&P Select Materials SPDR: Pivot points
SJM Holdings downgraded to Neutral from Outperform at Daiwa
First Republic price

In [12]:
b=final_tickerdf[final_tickerdf["Count"] >= 3 ]
final_ticker_more_3_df=b.sort_values(["Count","Tick"],ascending=False)
final_ticker_more_3_df.head()
# final_ticker_more_3_df.to_csv("todayflymorningnews_3ormore.csv",index=False)

,TickerNews,Tick,Count
226,Citizens Financial CEO reports 'strength in ou...,CFG,6
227,Citizens Financial reports Q4 net interest mar...,CFG,6
228,Citizens Financial reports Q4 underlying EPS 9...,CFG,6
229,Citizens Financial raises quarterly dividend 8...,CFG,6
230,Citizens Financial declares an 8% increase in ...,CFG,6


In [13]:
c=final_ticker_more_3_df["Tick"].unique()
ticksdf=pd.DataFrame(c) 
ticksdf
# ticksdf.to_csv("ticks.csv",index=False)

,0
0,CFG
1,STT
2,SLB
3,AAPL
4,SNAP
5,RF
6,MTCH
7,GPS
8,EFX
9,DRRX


In [15]:
ticker_dict={"TargetRaised":"target raised","Upgraded":"upgraded","Downgraded":"downgraded"}
final_tickerdf["TickerNews"]

226    Citizens Financial CEO reports 'strength in ou...
227    Citizens Financial reports Q4 net interest mar...
228    Citizens Financial reports Q4 underlying EPS 9...
229    Citizens Financial raises quarterly dividend 8...
230    Citizens Financial declares an 8% increase in ...
231    Citizens Financial reports Q4 EPS 99c, consens...
74         State Street up 4% after reporting Q4 results
75           State Street sees FY20 fee revenue up 1%-3%
76     State Street reports Q4 NII $636M vs. $697M la...
77     State Street reports Q4 AUM $3.12B vs. $2.51B ...
78     State Street reports Q4 EPS $1.73, consensus $...
175    Schlumberger reports Q4 cash flow from operati...
176    Schlumberger CEO: 'Recent geopolitical risk sh...
177    Schlumberger CEO says 'confident' in further c...
178    Schlumberger reports Q4 adjusted EPS 39c, cons...
141    Apple price target raised to $368 from $296 at...
142    Apple makes design change in 5G mmWave module ...
143    Netflix price target rai